In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm,f1_score,roc_curve,auc,classification_report


In [2]:
normal=pd.read_csv('type_nor.csv',names=["Number", "Time","Length","Type"],nrows=1054)
attack=pd.read_csv('type_att.csv',names=["Number", "Time","Length","Type"],nrows=1054)
#pd.read_csv("path/to/file.txt", 
              #    sep='\t', 
                #  names=["Sequence", "Start", "End", "Coverage"])
normal["Time"]=(normal["Time"]).diff().fillna(0.229507)
normal["Time"]

0       0.229507
1       0.166256
2       0.120978
3       0.360910
4       0.209079
          ...   
1049    0.328000
1050    0.445000
1051    0.398000
1052    0.285000
1053    0.417000
Name: Time, Length: 1054, dtype: float64

In [3]:
attack["Time"]=(attack["Time"]).diff().fillna(0.090555)
attack["Time"]

0       0.090555
1       0.041606
2       0.007822
3       0.007688
4       0.004262
          ...   
1049    0.007500
1050    0.055800
1051    0.045000
1052    0.000100
1053    0.009500
Name: Time, Length: 1054, dtype: float64

In [4]:
attack.shape

(1054, 4)

In [5]:
normal.shape

(1054, 4)

In [6]:
normal=normal.append(attack)
normal

,Number,Time,Length,Type
0,100,0.229507,162,ICMP
1,200,0.166256,180,ICMP
2,300,0.120978,174,ICMP
3,400,0.360910,170,ICMP
4,500,0.209079,164,ICMP
...,...,...,...,...
1049,105000,0.007500,170,ICMP
1050,105100,0.055800,171,ICMP
1051,105200,0.045000,170,ICMP
1052,105300,0.000100,169,ICMP


In [7]:
normal.iloc[:,3].value_counts()

ICMP      2002
Raw_IP     106
Name: Type, dtype: int64

In [8]:
ohe=pd.get_dummies(normal.iloc[:,3], drop_first=True)
ohe


,Raw_IP
0,0
1,0
2,0
3,0
4,0
...,...
1049,0
1050,0
1051,0
1052,0


In [9]:
normal=normal.iloc[:,:3]
normal

,Number,Time,Length
0,100,0.229507,162
1,200,0.166256,180
2,300,0.120978,174
3,400,0.360910,170
4,500,0.209079,164
...,...,...,...
1049,105000,0.007500,170
1050,105100,0.055800,171
1051,105200,0.045000,170
1052,105300,0.000100,169


In [10]:

X=pd.concat([normal,ohe],axis=1)
type(X)
X

,Number,Time,Length,Raw_IP
0,100,0.229507,162,0
1,200,0.166256,180,0
2,300,0.120978,174,0
3,400,0.360910,170,0
4,500,0.209079,164,0
...,...,...,...,...
1049,105000,0.007500,170,0
1050,105100,0.055800,171,0
1051,105200,0.045000,170,0
1052,105300,0.000100,169,0


In [11]:
normal_class = np.zeros(int((normal.shape[0])/2))
attack_class = np.ones(int((normal.shape[0])/2))

In [12]:
normal_class.shape

(1054,)

In [13]:
attack_class.shape

(1054,)

In [14]:
Y = np.append(normal_class,attack_class,axis=0)
Y.shape

(2108,)

In [15]:
X.values

array([[1.00000e+02, 2.29507e-01, 1.62000e+02, 0.00000e+00],
       [2.00000e+02, 1.66256e-01, 1.80000e+02, 0.00000e+00],
       [3.00000e+02, 1.20978e-01, 1.74000e+02, 0.00000e+00],
       ...,
       [1.05200e+05, 4.50000e-02, 1.70000e+02, 0.00000e+00],
       [1.05300e+05, 1.00000e-04, 1.69000e+02, 0.00000e+00],
       [1.05400e+05, 9.50000e-03, 1.69000e+02, 0.00000e+00]])

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size = 0.25, random_state = 0)

In [17]:
print(X_train)

[[7.450e+04 2.250e-02 1.680e+02 0.000e+00]
 [7.060e+04 9.000e-04 1.670e+02 0.000e+00]
 [3.270e+04 2.508e-01 1.720e+02 0.000e+00]
 ...
 [7.640e+04 3.710e-01 1.650e+02 0.000e+00]
 [8.360e+04 4.110e-01 1.750e+02 0.000e+00]
 [6.000e+04 2.340e-02 1.690e+02 0.000e+00]]


In [18]:
print(y_train)

[1. 1. 0. ... 0. 0. 1.]


In [19]:
print(X_test)

[[3.100e+04 6.000e-05 1.710e+02 0.000e+00]
 [1.000e+05 3.050e-02 1.710e+02 0.000e+00]
 [7.100e+03 3.159e-02 1.720e+02 0.000e+00]
 ...
 [5.200e+04 7.460e-02 1.720e+02 0.000e+00]
 [8.300e+03 5.646e-01 1.760e+02 0.000e+00]
 [1.044e+05 4.630e-02 1.690e+02 0.000e+00]]


In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
print(X_train)

[[ 0.71617157 -0.94214457 -0.40269411 -0.23688019]
 [ 0.58741292 -1.09312828 -0.63858125 -0.23688019]
 [-0.66385704  0.65366942  0.54085444 -0.23688019]
 ...
 [ 0.77890015  1.49386583 -1.11035552 -0.23688019]
 [ 1.01660843  1.77346531  1.24851586 -0.23688019]
 [ 0.23745351 -0.93585358 -0.16680697 -0.23688019]]


In [22]:
print(X_test)

[[-0.71998261 -1.09899987  0.30496731 -0.23688019]
 [ 1.55805506 -0.88622467  0.30496731 -0.23688019]
 [-1.50904203 -0.87860559  0.54085444 -0.23688019]
 ...
 [-0.0266668  -0.57796625  0.54085444 -0.23688019]
 [-1.46942399  2.84712728  1.484403   -0.23688019]
 [ 1.70332123 -0.77578288 -0.16680697 -0.23688019]]


In [23]:
X_train

array([[ 0.71617157, -0.94214457, -0.40269411, -0.23688019],
       [ 0.58741292, -1.09312828, -0.63858125, -0.23688019],
       [-0.66385704,  0.65366942,  0.54085444, -0.23688019],
       ...,
       [ 0.77890015,  1.49386583, -1.11035552, -0.23688019],
       [ 1.01660843,  1.77346531,  1.24851586, -0.23688019],
       [ 0.23745351, -0.93585358, -0.16680697, -0.23688019]])

In [24]:
X_test

array([[-0.71998261, -1.09899987,  0.30496731, -0.23688019],
       [ 1.55805506, -0.88622467,  0.30496731, -0.23688019],
       [-1.50904203, -0.87860559,  0.54085444, -0.23688019],
       ...,
       [-0.0266668 , -0.57796625,  0.54085444, -0.23688019],
       [-1.46942399,  2.84712728,  1.484403  , -0.23688019],
       [ 1.70332123, -0.77578288, -0.16680697, -0.23688019]])

# TREE

In [25]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [26]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [0. 0.]
 [1. 1.]]


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[263   1]
 [  2 261]]


0.9943074003795066

In [27]:
extra=pd.read_csv('type_att.csv',names=["Number", "Time","Length","Type"])
extra=extra.iloc[1055:,:]
extra.head()

,Number,Time,Length,Type
1055,105600,28.1585,167,ICMP
1056,105700,28.1586,169,ICMP
1057,105800,28.1662,169,ICMP
1058,105900,28.2641,173,ICMP
1059,106000,28.2652,166,ICMP


In [65]:
print(classifier.predict(sc.transform([[385800,.574,171,0]])))

[0.]


#  KNN

In [42]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
KNN.fit(X_train, y_train)

KNeighborsClassifier()

In [43]:
y_pred = KNN.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [0. 0.]
 [1. 1.]]


In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)
accuracy_score(y_test, y_pred)

[[261   3]
 [  1 262]]


0.9924098671726755

# svm

In [49]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', random_state = 0)
svm.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [50]:
y_pred = svm.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [0. 0.]
 [1. 1.]]


In [51]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)
accuracy_score(y_test, y_pred)

[[262   2]
 [  1 262]]


0.9943074003795066

# log

In [52]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(random_state = 0)
log.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [53]:
y_pred = log.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [0. 0.]
 [1. 1.]]


In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm5 = confusion_matrix(y_test, y_pred)
print(cm5)
accuracy_score(y_test, y_pred)

[[259   5]
 [  1 262]]


0.9886148007590133

# bayes

In [55]:
from sklearn.naive_bayes import GaussianNB
baye = GaussianNB()
baye.fit(X_train, y_train)

GaussianNB()

In [56]:
y_pred = baye.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1. 1.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 1.]
 [0. 0.]
 [1. 1.]]


In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm9 = confusion_matrix(y_test, y_pred)
print(cm9)
accuracy_score(y_test, y_pred)

[[255   9]
 [  1 262]]


0.9810246679316889